# Classification model - Saving Pickle File

In [1]:
import os
import re
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

### Load the data

In [2]:
data = pd.read_csv('ppp data.csv')
print(f"Shape of the data is: {data.shape}")
data.head()

Shape of the data is: (417266, 15)


,LoanAmount,City,State,Zip,NAICSCode,BusinessType,RaceEthnicity,Gender,Veteran,NonProfit,JobsReported,DateApproved,Lender,CD,LoanRange
0,133500,Pasadena,TX,77502.0,111110.0,Subchapter S Corporation,Unanswered,Unanswered,Unanswered,NaN,0.0,4/29/2020,"Capital One, National Association",TX-29,6
1,125300,Houston,TX,77041.0,111110.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,0.0,4/29/2020,Comerica Bank,TX-07,6
2,124938,PLANO,TX,75024.0,111110.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,9.0,4/14/2020,Veritex Community Bank,TX-03,5
3,118922,Sugar Land,TX,77478.0,111110.0,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,NaN,15.0,4/27/2020,Allegiance Bank,TX-22,5
4,109952,Houston,TX,77007.0,111110.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,17.0,4/28/2020,Allegiance Bank,TX-18,5


### Check how many `null/Nan` values are in each column

In [3]:
data.isnull().sum()

LoanAmount         0
City              14
State              0
Zip                0
NAICSCode       9162
BusinessType     258
DateApproved       0
Lender             0
CD                18
LoanRange          0
dtype: int64

### View the possible labels for the columns that have some null values

In [4]:
# # we are intentionally narrowing the scope of the loan data details for those with strong completion entries, minimal nulls
# contains_null = ['Zip', 'BusinessType', 'DateApproved', 'Lender','CD', 'LoanRange']

# for col in contains_null:
#     print(f"List of unique labels for {col}:::{set(data[col])}")

### Clean up null values

For the sake of simplicity, I'm going to make a few assumptions:

- `Dependents`: Assumption that there are no dependents (0: 345 | 1: 102 | 2: 101 | 3+: 51)
- `Self_Employed`: Assumption that the applicant is not self-employed (No: 500 | Yes: 82)
- `Credit_History`: Assumption that the person has a credit history (True: 475 | False: 89)
- `Married`: If nothing specified, applicant is not married
- `Gender`: Assuming the gender is Male for the missing values (Male: 489 | Female: 112)

In [5]:
# data['Dependents'] = data['Dependents'].fillna('0')
# data['Self_Employed'] = data['Self_Employed'].fillna('No')
# data['Credit_History'] = data['Credit_History'].fillna(1)
# data['Married'] = data['Married'].fillna('No')
# data['Gender'] = data['Gender'].fillna('Male')

In [6]:
# investigate rows with null values
data[data.isnull().any(axis=1)]

,LoanAmount,City,State,Zip,NAICSCode,BusinessType,DateApproved,Lender,CD,LoanRange
2863,37053,WALLIS,TX,77485,111998.0,NaN,5/3/2020,"Wells Fargo Bank, National Association",TX-10,2
5759,21396,FLORESVILLE,TX,78114,112990.0,NaN,5/3/2020,"Wells Fargo Bank, National Association",TX-28,1
16293,15102,ROSENBERG,TX,77471,236115.0,NaN,5/3/2020,"Wells Fargo Bank, National Association",TX-22,1
28793,45001,DALLAS,TX,75228,238160.0,NaN,5/1/2020,"JPMorgan Chase Bank, National Association",TX-05,2
30297,136475,KENNEDALE,TX,76060,238210.0,NaN,5/1/2020,"JPMorgan Chase Bank, National Association",TX-06,6
...,...,...,...,...,...,...,...,...,...,...
416999,"e $150,000-350,000",Berkeley,TX,75081,NaN,Limited Liability Company(LLC),5/1/2020,"JPMorgan Chase Bank, National Association",TX-32,7
417101,"e $150,000-350,000",RIVERSIDE,TX,75247,NaN,Corporation,5/1/2020,"JPMorgan Chase Bank, National Association",TX-30,7
417135,"e $150,000-350,000",REEDLEY,TX,78753,NaN,Corporation,5/1/2020,"JPMorgan Chase Bank, National Association",TX-10,7
417154,"e $150,000-350,000",Yucaipa,TX,77598,NaN,Corporation,5/1/2020,"JPMorgan Chase Bank, National Association",TX-22,7


In [7]:
# only take rows of businesses that CD is not null

data = data[data['CD'].notna()]
data.head()

#drop rows with NA cities
data = data[data['City'].notna()]

# drop rows with NA NAICS code
data = data[data['NAICSCode'].notna()]

#drop rows with NA business type
data = data[data['BusinessType'].notna()]

data.isnull().sum()

data.shape

data.head()

data['NAICSCode'] = data['NAICSCode'].astype(int)

data

,LoanAmount,City,State,Zip,NAICSCode,BusinessType,DateApproved,Lender,CD,LoanRange
0,133500,Pasadena,TX,77502,111110,Subchapter S Corporation,4/29/2020,"Capital One, National Association",TX-29,6
1,125300,Houston,TX,77041,111110,Corporation,4/29/2020,Comerica Bank,TX-07,6
2,124938,PLANO,TX,75024,111110,Corporation,4/14/2020,Veritex Community Bank,TX-03,5
3,118922,Sugar Land,TX,77478,111110,Limited Liability Company(LLC),4/27/2020,Allegiance Bank,TX-22,5
4,109952,Houston,TX,77007,111110,Corporation,4/28/2020,Allegiance Bank,TX-18,5
...,...,...,...,...,...,...,...,...,...,...
417248,"e $150,000-350,000",Lafayette,TX,77064,541611,Limited Liability Company(LLC),5/1/2020,"JPMorgan Chase Bank, National Association",TX-07,7
417249,"e $150,000-350,000",SANTA ROSA,TX,78205,531312,Corporation,4/14/2020,Vantage Bank Texas,TX-35,7
417250,"e $150,000-350,000",RANCHO CORDOVA,TX,77803,442210,Limited Liability Company(LLC),4/27/2020,"Spirit of Texas Bank, SSB",TX-17,7
417251,"e $150,000-350,000",LOS ALTOS,TX,78230,492210,Corporation,4/30/2020,Randolph-Brooks FCU,TX-21,7


### View cleaned up values

In [8]:
# label_cols = ['LoanAmount', 'City', 'State', 'Zip', 'NAICSCode', 'BusinessType', 'DateApproved', 'Lender', 'CD', 'LoanRange']

# for col in label_cols:
#     print(f"List of unique labels for {col} ::: {set(data[col])}")

In [9]:
## read in data for NAICS code

naics = pd.read_csv('US-Business-Profiles-By-Sales-and-Employees (1).csv')

naics.head()


,NAICS 1 Code,NAICS 1 Description
0,111110,Soybean Farming
1,111120,Oilseed (except Soybean) Farming
2,111130,Dry Pea and Bean Farming
3,111140,Wheat Farming
4,111150,Corn Farming


In [16]:
naics.dtypes
naics[naics.isnull().any(axis=1)]


naics['NAICS 1 Code'] = naics['NAICS 1 Code'].astype(int)

naics

,NAICS 1 Code,NAICS 1 Description
0,111110,Soybean Farming
1,111120,Oilseed (except Soybean) Farming
2,111130,Dry Pea and Bean Farming
3,111140,Wheat Farming
4,111150,Corn Farming
...,...,...
1040,926140,Regulation of Agricultural Marketing and Commo...
1041,926150,"Regulation, Licensing, and Inspection of Misce..."
1042,927110,Space Research and Technology
1043,928110,National Security


In [18]:
## merge tables 

pd.merge(data, naics, left_on='NAICSCode', right_on='NAICS 1 Code')

,LoanAmount,City,State,Zip,NAICSCode,BusinessType,DateApproved,Lender,CD,LoanRange,NAICS 1 Code,NAICS 1 Description
0,133500,Pasadena,TX,77502,111110,Subchapter S Corporation,4/29/2020,"Capital One, National Association",TX-29,6,111110,Soybean Farming
1,125300,Houston,TX,77041,111110,Corporation,4/29/2020,Comerica Bank,TX-07,6,111110,Soybean Farming
2,124938,PLANO,TX,75024,111110,Corporation,4/14/2020,Veritex Community Bank,TX-03,5,111110,Soybean Farming
3,118922,Sugar Land,TX,77478,111110,Limited Liability Company(LLC),4/27/2020,Allegiance Bank,TX-22,5,111110,Soybean Farming
4,109952,Houston,TX,77007,111110,Corporation,4/28/2020,Allegiance Bank,TX-18,5,111110,Soybean Farming
...,...,...,...,...,...,...,...,...,...,...,...,...
397943,"d $350,000-1 million",PHOENIX,TX,79072,325193,Limited Liability Company(LLC),4/15/2020,"JPMorgan Chase Bank, National Association",TX-19,8,325193,Ethyl Alcohol Manufacturing
397944,"d $350,000-1 million",TUCSON,TX,79045,325193,Limited Liability Company(LLC),4/15/2020,"JPMorgan Chase Bank, National Association",TX-19,8,325193,Ethyl Alcohol Manufacturing
397945,"d $350,000-1 million",Scottsdale,TX,75034,325193,Limited Liability Company(LLC),4/10/2020,"JPMorgan Chase Bank, National Association",TX-26,8,325193,Ethyl Alcohol Manufacturing
397946,"d $350,000-1 million",YUMA,TX,77002,212113,Limited Liability Company(LLC),4/9/2020,Gulf Capital Bank,TX-18,8,212113,Anthracite Mining


### Encode categorical fields
We have a lot of `string` labels that we encounter in `Gender`, `Married`, `Education`, `Self_Employed` & `Property_Area` columns.

In [ ]:
# create dictionaries to map fields to numeric values
gender_values = {'Female' : 0, 'Male' : 1} 
married_values = {'No' : 0, 'Yes' : 1}
education_values = {'Graduate' : 0, 'Not Graduate' : 1}
employed_values = {'No' : 0, 'Yes' : 1}
dependent_values = {'3+': 3, '0': 0, '2': 2, '1': 1}
target_values = {'Y':1, 'N':0}

# replace values in each column according to the dictionaries above
data.replace({'Gender': gender_values, 'Married': married_values, 'Education': education_values, \
                'Self_Employed': employed_values, 'Dependents': dependent_values, 'Loan_Status': target_values}, inplace=True)

# Get dummy variables for nominal property column
clean_data = pd.get_dummies(data, columns=["Property_Area"])

clean_data.head()

### Train/Test Split

In [ ]:
# store the target variable in y and everything else goes in X
y = clean_data['Loan_Status']

# we're also dropping Load_ID because it adds no value to the prediction
X = clean_data.drop(['Loan_ID', 'Loan_Status'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)

X_train.shape

### Fill the nulls for the continuous variables with the mean values of that column

In [ ]:
X_train['Loan_Amount_Term'] = X_train['Loan_Amount_Term'].fillna(X_train['Loan_Amount_Term'].mean())
X_train['LoanAmount'] = X_train['LoanAmount'].fillna(X_train['LoanAmount'].mean())
X_test['Loan_Amount_Term'] = X_test['Loan_Amount_Term'].fillna(X_test['Loan_Amount_Term'].mean())
X_test['LoanAmount'] = X_test['LoanAmount'].fillna(X_test['LoanAmount'].mean())


# View the datatypes of all columns
X_train.dtypes

### Confirm that we no longer have any nulls

In [ ]:
X_train.isnull().sum()

### Create the classifier model and the parameter grid for GridSearch

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

param_grid = {"n_estimators" : [10, 20, 50, 100],
             "max_depth" : [None, 6, 8, 10],
             "max_leaf_nodes": [None, 5, 10, 20], 
             "min_impurity_split": [0.1, 0.2, 0.3, 0.4]}

### Initialize the GridSearch to tune my hyperparameters

In [ ]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(model, param_grid=param_grid, cv=3, verbose=2)

### Execute the tuning

In [ ]:
grid.fit(X_train, y_train)

### View the results

In [ ]:
print(f"Best parameters: {grid.best_params_}")
print(f"Test set score: {grid.score(X_test, y_test)}")

# Saving the model

In [ ]:
import pickle

with open('RF_model.pkl','wb') as f:
    pickle.dump(grid, f)

### Loading the saved model

In [ ]:
with open('RF_model.pkl', 'rb') as f:
    model = pickle.load(f)

### Test the saved model

In [ ]:
print(f"Test set score: {model.score(X_test, y_test)}")

In [ ]:
model